In [ ]:
import os 
import glob
import pandas as pd
from ics import Calendar, Event
from dataclasses import dataclass, field 
from collections import defaultdict
from typing import List, Dict, Optional
import numpy as np
from datetime import datetime

: 

In [2]:
import logging
logging.basicConfig()
LOGGER = logging.getLogger(__name__)
LOGGER.setLevel(logging.INFO)


In [88]:
import json
raw_auto_export = 'source/HealthAutoExport-2023-01-16-2023-01-16.json'
with open(raw_auto_export, 'r') as ae: 
    raw_data = json.loads(ae.read())
    
date_updated = raw_auto_export.split('.json')[0].split('source/HealthAutoExport-')[1]

In [93]:
def unnest_data_points(data: Dict) -> List[Dict]:
    """
    Unnest data points from a single metric.
    """
    unnest_rows = [] 
    prefix = data['name']
    nested_unit = data['units']
    for type, qty in data.items(): 
        point = {} 
        col_name = f"{prefix}_{type}"
        
        point['name'] = col_name
        point['unit'] = nested_unit
        point['qty'] = qty
        unnest_rows.append(point)
    print(unnest_rows)
    return unnest_rows
        

In [95]:
rows = []
for metric in raw_data.get("data", {}).get("metrics", []):
    name = metric["name"]
    units = metric["units"]

    for point in metric.get("data", []):
        point["name"] = name
        point["units"] = units
        has_qty = point.get('qty')
        if has_qty: 
            rows.append(point)
        else: 
            unnest_data_points(point)
            rows.extend(unnest_data_points(point))

# return rows

[{'name': 'sleep_analysis_sleepStart', 'unit': 'hr', 'qty': '2023-01-15 23:46:00 +1100'}, {'name': 'sleep_analysis_sleepSource', 'unit': 'hr', 'qty': 'AutoSleep'}, {'name': 'sleep_analysis_inBedEnd', 'unit': 'hr', 'qty': '2023-01-16 07:19:22 +1100'}, {'name': 'sleep_analysis_date', 'unit': 'hr', 'qty': '2023-01-16 06:37:52 +1100'}, {'name': 'sleep_analysis_inBedStart', 'unit': 'hr', 'qty': '2023-01-15 23:40:22 +1100'}, {'name': 'sleep_analysis_asleep', 'unit': 'hr', 'qty': 7.216666666666667}, {'name': 'sleep_analysis_inBedSource', 'unit': 'hr', 'qty': 'Nam’s Apple\xa0Watch'}, {'name': 'sleep_analysis_inBed', 'unit': 'hr', 'qty': 14.808333333333334}, {'name': 'sleep_analysis_sleepEnd', 'unit': 'hr', 'qty': '2023-01-16 07:18:00 +1100'}, {'name': 'sleep_analysis_name', 'unit': 'hr', 'qty': 'sleep_analysis'}, {'name': 'sleep_analysis_units', 'unit': 'hr', 'qty': 'hr'}]
[{'name': 'sleep_analysis_sleepStart', 'unit': 'hr', 'qty': '2023-01-15 23:46:00 +1100'}, {'name': 'sleep_analysis_sleepSo

In [96]:
rows

[{'source': None,
  'qty': 854.7099999999991,
  'date': '2023-01-16 00:00:00 +1100',
  'name': 'active_energy',
  'units': 'kcal'},
 {'date': '2023-01-16 00:00:00 +1100',
  'source': None,
  'qty': 129,
  'name': 'apple_exercise_time',
  'units': 'min'},
 {'date': '2023-01-16 00:00:00 +1100',
  'qty': 12,
  'name': 'apple_stand_hour',
  'units': 'count'},
 {'qty': 206.30664193929996,
  'date': '2023-01-16 00:00:00 +1100',
  'source': None,
  'name': 'carbohydrates',
  'units': 'g'},
 {'qty': 165.27904518309998,
  'date': '2023-01-16 00:00:00 +1100',
  'source': None,
  'name': 'protein',
  'units': 'g'},
 {'name': 'sleep_analysis_sleepStart',
  'unit': 'hr',
  'qty': '2023-01-15 23:46:00 +1100'},
 {'name': 'sleep_analysis_sleepSource', 'unit': 'hr', 'qty': 'AutoSleep'},
 {'name': 'sleep_analysis_inBedEnd',
  'unit': 'hr',
  'qty': '2023-01-16 07:19:22 +1100'},
 {'name': 'sleep_analysis_date',
  'unit': 'hr',
  'qty': '2023-01-16 06:37:52 +1100'},
 {'name': 'sleep_analysis_inBedStart',


In [90]:
rows

[{'source': None,
  'qty': 854.7099999999991,
  'date': '2023-01-16 00:00:00 +1100',
  'name': 'active_energy',
  'units': 'kcal'},
 {'date': '2023-01-16 00:00:00 +1100',
  'source': None,
  'qty': 129,
  'name': 'apple_exercise_time',
  'units': 'min'},
 {'date': '2023-01-16 00:00:00 +1100',
  'qty': 12,
  'name': 'apple_stand_hour',
  'units': 'count'},
 {'qty': 206.30664193929996,
  'date': '2023-01-16 00:00:00 +1100',
  'source': None,
  'name': 'carbohydrates',
  'units': 'g'},
 {'qty': 165.27904518309998,
  'date': '2023-01-16 00:00:00 +1100',
  'source': None,
  'name': 'protein',
  'units': 'g'},
 {'sleepStart': '2023-01-15 23:46:00 +1100',
  'sleepSource': 'AutoSleep',
  'inBedEnd': '2023-01-16 07:19:22 +1100',
  'date': '2023-01-16 06:37:52 +1100',
  'inBedStart': '2023-01-15 23:40:22 +1100',
  'asleep': 7.216666666666667,
  'inBedSource': 'Nam’s Apple\xa0Watch',
  'inBed': 14.808333333333334,
  'sleepEnd': '2023-01-16 07:18:00 +1100',
  'name': 'sleep_analysis',
  'units': '

In [27]:
# Functions
def collect_event_stats(stats_df: pd.DataFrame, column_names: List[str]) -> pd.DataFrame: 
    """
    Extract health data from stats dataframe in the format 
    [['name', 'qty']] where: 
    - name: name of the health data
    - qty: value (in respective unit) of the health data
    
    Output: 
        - DataFrame of health data in the format [['name', 'qty']]
    """
    filtered_stats = stats_df[stats_df['name'].isin(column_names)]
    event_type_stats  = filtered_stats[['name', 'qty']]
    
    return event_type_stats

def create_day_events(stats: pd.DataFrame, event_date: str) -> List[Event]: 
    """
    Iterate through different event types (food / activity / sleep)
    and generate events to add to the daily calendar only if event exists
    """
    day_events = []
    
    for types, col_names in EVENT_TYPES.items(): 
        # collect object name and arguments
        # dynamically create event type objects 
        dataclass_name = types
        dataclass_obj = globals()[dataclass_name]
        
        # collect object arguments to initialise objects from stats
        dataclass_obj_stats = collect_event_stats(stats_df=stats, column_names=col_names)
        
        obj_args = dict(dataclass_obj_stats.values)
        
        if obj_args: 
            obj = dataclass_obj(**obj_args)
            e = AppleHealthEvent(
                date = event_date,
                title = obj.title, 
                description = obj.description
            ).event
            day_events.append(e)
            
    return day_events


def convert_kj_to_cal(row, new_name):
    """Converts kj to calories"""
    row_dict = row.to_dict()
    calorie_value = int(row['qty']/4)
    row_dict['qty'] = calorie_value
    row_dict['name'] = new_name
    row_dict['units'] = 'kcal'
    
    return pd.DataFrame(row_dict, index=[0])


def convert_to_12_hr(time_str: str) -> str:
    "Lambda function to convert 24 hour time to 12 hour time"
    time_as_24_hr = datetime.strptime(time_str, "%Y-%m-%d %H:%M:%S %z").time()
    time_as_12_hr = time_as_24_hr.strftime("%-I:%M %p")
    return time_as_12_hr


In [3]:
import json
with open("config/user_config.json", "r") as f:
    user_config = json.load(f)

analysis_cols = ["qty", "dates", "name", "units"]
raw_data_columns = user_config.get("raw_data_columns")
EVENT_TYPES = user_config.get("event_types")


In [25]:
base_folder = os.getcwd()
source_folder = base_folder + '/source/'
apple_health_files = glob.glob(source_folder + '*.json')
file_names = [os.path.basename(x) for x in apple_health_files]

df_raw = pd.DataFrame()
# read configs
with open("config/user_config.json", "r") as f:
    user_config = json.load(f)

analysis_cols = ["qty", "dates", "name", "units"]
raw_data_columns = user_config.get("raw_data_columns")
EVENT_TYPES = user_config.get("event_types")


# read in raw data 
for json_file in apple_health_files:
    json_raw = pd.read_json(json_file, lines = True)
    df_raw = pd.concat([df_raw, json_raw])

In [29]:
df_raw

,source,date,qty,name,units,Avg,Max,Min,sleepStart,asleep,inBed,sleepEnd,sleepSource,inBedStart,inBedEnd,inBedSource
0,NaN,2023-01-10 00:00:00+11:00,749.722000,active_energy,kcal,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,NaN,2023-01-11 00:00:00+11:00,656.360000,active_energy,kcal,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,NaN,2023-01-12 00:00:00+11:00,957.423000,active_energy,kcal,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,NaN,2023-01-13 00:00:00+11:00,507.229000,active_energy,kcal,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,NaN,2023-01-14 00:00:00+11:00,667.169000,active_energy,kcal,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
384,NaN,2023-01-12 00:00:00+11:00,9.817352,zinc,mg,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
385,NaN,2023-01-13 00:00:00+11:00,2.635352,zinc,mg,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
386,NaN,2023-01-14 00:00:00+11:00,4.277020,zinc,mg,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
387,NaN,2023-01-15 00:00:00+11:00,3.408752,zinc,mg,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [28]:
df_ahc = df_raw.copy()

# define transformations to go from df_raw to df_ahc (apple-health-calendar)
# cleaning values
df_ahc["dates"] = pd.to_datetime(df_ahc["date"]).dt.date
df_ahc["qty"] = df_ahc["qty"].fillna(df_ahc["asleep"])

# create calories
logging.info((df_ahc[df_ahc["name"] == "active_energy"]))
active_energy_rows = df_ahc[df_ahc["name"] == "active_energy"][analysis_cols]

for _, row in active_energy_rows.iterrows():
    df_row = convert_kj_to_cal(row, "calories_burnt")
    df_ahc = pd.concat([df_ahc, df_row])

# unpivot sleep columns into its own
df_sleep = df_ahc[df_ahc["name"].isin(["sleep_analysis"])]
sleep_data = df_sleep[["asleep", "inBed", "inBedStart", "dates"]].reset_index(
    drop=True
)

# convert inBedStart to 12 hour time into new column inBedStartTime
sleep_data["inBedStartTime"] = sleep_data.apply(
    lambda row: convert_to_12_hr(row["inBedStart"]), axis=1
)

# Lambda functions (apply functions by rows)
df_sleep_data = pd.melt(
    sleep_data, id_vars=["dates"], value_vars=["asleep", "inBed", "inBedStartTime"]
).rename(columns={"variable": "name", "value": "qty"})

# merge back to original data
df_ahc = pd.concat([df_ahc, df_sleep_data])

# filter out values
df_ahc = df_ahc[df_ahc["name"].isin(raw_data_columns)][analysis_cols].reset_index(
    drop=True
)

# CHECKPOINT
df_ahc.to_csv("outputs/transformations/ahc.csv", index=False)

c = Calendar()
available_dates = df_ahc['dates'].unique()

weekly_events = []
for date in available_dates:
    LOGGER.info("Generating events for %s", date)
    daily_stats = df_ahc[df_ahc["dates"] == date]
    daily_events = create_day_events(stats=daily_stats, event_date=date)
    weekly_events.append(daily_events)

INFO:__main__:Generating events for 2023-01-10
INFO:__main__:Generating events for 2023-01-11
INFO:__main__:Generating events for 2023-01-12
INFO:__main__:Generating events for 2023-01-13
INFO:__main__:Generating events for 2023-01-14
INFO:__main__:Generating events for 2023-01-15
INFO:__main__:Generating events for 2023-01-16


In [20]:
@dataclass 
class AppleHealthEvent(Event):
    """
    An event derived from Apple Health data
    For usage within .ics format 
    """
    date: datetime.date
    description: str 
    title: str
            
    @property 
    def event(self):
        all_day_date = f"{self.date} 00:00:00"
        e = Event()
        e.name = self.title
        e.description = self.description
        e.begin = all_day_date
        e.end = all_day_date
        e.make_all_day()

        return e

In [21]:
@dataclass
class Time: 
    "A basic time object in hours"
    time: Optional[float] = field(default=0)
    timeInMinutes: Optional[float] = field(default=0)
        
    def __post_init__(self): 
        if self.timeInMinutes:
            self.time = self.timeInMinutes / 60
            
        
    @property 
    def hours(self) -> float: 
        hours = int(self.time)
        return hours 
    
    @property
    def minutes(self) -> str:
        minutes = int((self.time - self.hours) * 60)
        return minutes
    
    @property
    def title(self) -> str:
        title = f"{self.hours}h {self.minutes}m"
        return title

@dataclass 
class Food: 
    "A basic food object"
    carbohydrates: float 
    protein: float 
    total_fat: float 
    fiber: float 
    calories_burnt: float

    def __post_init__(self): 
        # rename objects for easier usage 
        self.carb = self.carbohydrates
        self.fat = self.total_fat
    
    @property 
    def calories_ate(self) -> float: 
        calories = (self.carb + self.protein) * 4 + (self.fat) * 9
        return calories
    
    @property 
    def macros(self) -> str: 
        return f"{self.carb:.0f}C, {self.protein:.0f}P, {self.fat:.0f}F"

    @property 
    def title(self) -> str: 
        title = f"🔥 {self.calories_ate:.0f} cals ({self.macros})" 
        return title 
    
    @property 
    def description(self) -> str: 
        description = f"""
        🔥 {self.calories_burnt:.0f} kcal
        🍽️ {self.calories_ate:.0f} kcal
        🥞 {self.macros} 
        🍇 {self.fiber:.0f} g
        """
        return description

@dataclass
class Activity: 
    "A basic activity for activity and mindfulness"
    apple_exercise_time: Time
    mindful_minutes: Time = None
    
    def __post_init__(self): 
        # rename objects for easier usage
        self.apple_exercise_time = Time(timeInMinutes=self.apple_exercise_time)
        self.mindful_minutes = Time(timeInMinutes=self.mindful_minutes)
    
    @property 
    def activity_description(self) -> str: 
        a_description = f"🚴‍♂️ Activity: {self.apple_exercise_time.title} active" 
        return a_description

    @property 
    def mindful_description(self) -> str:
        m_description = f"🧘 Mindful: {self.mindful_minutes.title} mindful"
        return m_description
     
    @property 
    def description(self) -> str: 
        description = f"""
        {self.activity_description}
        {self.mindful_description}
        """
        return description
    
    @property 
    def mindful_title(self) -> str: 
        title = f"🧠 {self.mindful_minutes.minutes} mins "
        return title
    
    @property 
    def activity_title(self) -> str: 
        "Create blocks of 1 hour increments of activity minutes"
        block = str(self.apple_exercise_time.hours)
        title = f"🚴‍♂️ {block}"
        return title 
  
    @property 
    def title(self) -> str: 
        title = f"{self.mindful_title} | {self.activity_title}"
        return title 
   


@dataclass
class Sleep:
    "A basic sleep object"
    asleep: Time
    inBed: Time
    inBedStartTime: str

    def __post_init__(self): 
        # rename objects for easier usage
        self.time_asleep = Time(time=self.asleep)
        self.time_in_bed = Time(self.inBed)
        self.in_bed_time = self.inBedStartTime
    
    @property 
    def efficiency(self) -> float: 
        efficiency = self.time_asleep.time / self.time_in_bed.time * 100
        return efficiency
    
    @property
    def efficiency_title(self) -> str: 
        efficient = f"{self.efficiency:.0f}%"
        efficiency_title = f"🛏️ {efficient}"
        return efficiency_title
    
    @property 
    def title(self) -> str: 
        title = f"💤 {self.time_asleep.title} ({self.in_bed_time})"
        print(title)
        return title 
    
    @property
    def description(self) -> str: 
        s_description = f"""
        💤 Time asleep: {self.time_asleep.title}
        🛏️ Time in bed: {self.time_in_bed.title}
        🧮 Efficiency: {self.efficiency_title}
        """
        return s_description


In [269]:
c = Calendar()
available_dates = ahc['dates'].unique()

for date in available_dates:
    daily_stats = ahc[ahc['dates'] == date]    
    daily_calendar = create_day_events(
        stats=daily_stats,
        event_date=date
    )
    for event in daily_calendar:
        c.events.add(event)
        
        
with open('outputs/apple_health.ics', 'w') as calendar_file:
    calendar_file.writelines(c)

🧠 0 | 🚴‍♂️ 1
💤 9h 29m (9:16 PM)
🧠 0 | 🚴‍♂️ 2
💤 7h 16m (10:23 PM)
🧠 0 | 🚴‍♂️ 1
💤 8h 30m (8:01 PM)
🧠 0 | 🚴‍♂️ 1
💤 9h 1m (9:15 PM)
🧠 0 | 🚴‍♂️ 2
🧠 0 | 🚴‍♂️ 1
💤 6h 28m (9:31 PM)
🧠 0 | 🚴‍♂️ 0
💤 7h 35m (9:02 PM)


In [55]:
ics_file_name = 'outputs/apple-health.ics'
with open(ics_file_name, 'w') as f:
    f.write(str(c))
    f.close()

/Users/ntonthat/Library/Caches/pypoetry/virtualenvs/apple-health-calendar-FpPaTjHZ-py3.11/lib/python3.11/site-packages/ics/component.py:85: FutureWarning: Behaviour of str(Component) will change in version 0.9 to only return a short description, NOT the ics representation. Use the explicit Component.serialize() to get the ics representation.
  warnings.warn(
